# Cycloid Fitting

### Simplified to reduce notebook size

## Load Cyloid Data Points and Bezier Curves

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import curves.bezier as bezier
import curves.fitCurves as fit
import StressTools as tools
import utils
import fitting
import cycloidData
from scipy import stats
from os import path
from datetime import datetime

interior = utils.import_interior('interior1')

TOLERANCE = 1

cycloids = cycloidData.load_all_cycloids()
highResCycloids = cycloidData.load_all_cycloids(points_per_curve=1000)

min_vals = np.array([0, 0.1, 0])
max_vals = np.array([360, 1, 360])
constraints = [
    dict(wrapValue=True, minValue=1e-8, maxValue=1),
    dict(minValue=1e-8, maxValue=1, unstick=True),
    dict(wrapValue=True, minValue=0, maxValue=1)
]

tight_obliquity_contstraints = [
        dict(wrapValue=True, minValue=1e-8, maxValue=1),
        dict(minValue=0.62, maxValue=0.82, unstick=True),
        dict(wrapValue=True, minValue=0, maxValue=1)
    ]

def getConstraints(obliquity, longitude_max=1):
    return [
        dict(wrapValue=True, minValue=1e-8, maxValue=1),
        dict(minValue=obliquity, maxValue=obliquity, unstick=False),
        dict(wrapValue=True, minValue=0, maxValue=longitude_max)
    ]

def get_longitude_only_constraints(obliquity, phase):
    return [
        dict(minValue=phase, maxValue=phase, unstick=False),
        dict(minValue=obliquity, maxValue=obliquity, unstick=False),
        dict(wrapValue=True, minValue=0, maxValue=1)
    ]



### Helper Functions

In [2]:
def logmessage(msg):
    now = datetime.now()
    timestamp = now.strftime("%H:%M:%S")
    
    print(f'[{timestamp}] {msg}')
    

def translate_params(params, minVals, maxVals):
    if len(params) == 3:
        variables = params * (max_vals - min_vals) + min_vals # denormalize
    else:
        variables = params * (max_vals[0:2:] - min_vals[0:2:]) + min_vals[0:2:]

    return variables

def setChartXLimit(points, plt):
    BUFFER_PERCENT = 0.025

    first = points['lon'].max()
    last = points['lon'].min()

    buffer = (first - last) * BUFFER_PERCENT

    plt.xlim(first + buffer, last - buffer)


def check_fit(params, minVals, maxVals, curve, interior, tolerance=0.25, title='',
              verbose=True, path='./output/stressfield.csv.gz'):

    if len(params) == 3:
        variables = params * (max_vals - min_vals) + min_vals # denormalize
    else:
        variables = params * (max_vals[0:2:] - min_vals[0:2:]) + min_vals[0:2:]

    data, loss = fitting.match_stresses(curve,
                                        variables,
                                        interior,
                                        save_stress_field=True,
                                        path=path)
    if verbose:
        plt.figure()
        plt.title(f'{title} - Orientation Match')
        fit_points = data.loc[data['deltaHeading'] < tolerance].copy()

        if len(variables) >= 3:
            fit_points['lon'] = fit_points['lon'] - variables[2]

        plt.plot(curve['lon'], curve['lat'])
        setChartXLimit(curve, plt)

        plt.scatter(fit_points['lon'], fit_points['lat'], alpha=0.3, color='green')

        plt.figure()
        plt.title(f'{title} - Stress Magnitude')
        plt.scatter(fit_points['pointNumber'], fit_points['stress'])

        print(np.array(variables))

    return data

def plot_time(data):
    timeData = data.copy()
    time = np.array(timeData['time'])
    time[time < 180] = time[time < 180] + 360
    timeData['time'] = time

    plt.figure()
    plt.scatter(timeData['pointNumber'], timeData['time'],s=1)
    plt.title("Time")

def analyze_fit(opt, curve, tolerance, name=''):
    plt.figure()
    plt.title('Optimizer Loss Values')
    plt.plot(opt[0])

    display(opt[1])
    display(opt[2])

    params = opt[1]['parameters']
    bestCase = check_fit(params, min_vals, max_vals, curve, interior, tolerance=tolerance,
                         title=f'{name} Best Fit')

    params = opt[2]['parameters']
    finalCase = check_fit(params, min_vals, max_vals, curve, interior, tolerance=tolerance,
                         title=f'{name} Final Fit')


    # Plot time progression
    plot_time(bestCase)

    return bestCase, finalCase

def analyze_params(params, curve, tolerance, name='', verbose=True, folder='./output/', suffix=''):
    fieldPath = path.join(folder, f'{name}{suffix}StressField.csv.gz')
    results = check_fit(params, min_vals, max_vals, curve, interior, tolerance=tolerance,
                       title=f'{name} Highest Probability Fit', verbose=verbose, path=fieldPath)

    if verbose:
        plot_time(results)

    return results

def direct_fit(curve, tolerance, show_plots=True, params=[0, 0, 0]):
    data, loss = fitting.match_stresses(curve, params, interior)
    fit_points = data.loc[data['deltaHeading'] < tolerance].copy()

    if show_plots:
        plt.figure()
        plt.plot(curve['lon'], curve['lat'])
        setChartXLimit(curve, plt)

        plt.scatter(fit_points['lon'], fit_points['lat'], alpha=0.3, color='green')
        plt.figure()
        plt.scatter(fit_points['pointNumber'], fit_points['stress'])

        plot_time(data)

    return data, loss

def process_cycloid(curve,
                    name,
                    paramCount = 2,
                    analysisCurve=None,
                    folder='./output/',
                    iterations=500,
                    constraints=tight_obliquity_contstraints,
                    verbose=True):

    logmessage(f'Processing Cycloid: {name}')
    plt.close('all')
    numParams = paramCount
    start_params = [np.random.rand() for iter in range(numParams)]
    start_params[1] = 0.67

    optimizer = fitting.Adam(alpha=0.05)

    opt = optimizer.minimize(
        fitting.test_stress_parameters,
        curve,
        start_params,
        interior,
        constraints=constraints,
        max_iterations=iterations,
        verbose=verbose,
        batch_size=16
    )

    params = fitting.find_best_parameters(opt)
    fullCurve = analysisCurve if analysisCurve is not None else curve
    
    logmessage(f'Analyzing optimization parameters for {name}')
    bestFit = analyze_params(np.array(params), fullCurve, 0.25, name, verbose, folder)

    cols = ['loss', 'phase', 'obliquity']
    df = pd.DataFrame(opt[3], columns=cols).copy()

    df['phase'] = df['phase'] * (max_vals[0] - min_vals[0]) + min_vals[0]
    df['obliquity'] = df['obliquity'] * (max_vals[1] - min_vals[1]) + min_vals[1]

    df.to_csv(f'{folder}{name}Fits.csv.gz', index=False, encoding='utf-8', compression='gzip')
    bestFit.to_csv(f'{folder}{name}BestFit.csv.gz', index=False, encoding='utf-8', compression='gzip')
    
def process_cycloid_by_name(name, folder='./output/', iterations=5000, constraints=constraints, verbose=False):
    process_cycloid(cycloids[name].curve,
                    name,
#                     analysisCurve=highResCycloids[name].curve,
                    folder=folder,
                    iterations=iterations,
                    constraints=constraints,
                    verbose=verbose)

def process_cycloid_top_fits(curve,
                    name,
                    paramCount = 2,
                    analysisCurve=None,
                    folder='./output/',
                    iterations=500,
                    constraints=constraints,
                    verbose=True,
                    number_of_fits=5):

    numParams = paramCount
    start_params = [np.random.rand() for iter in range(numParams)]
    start_params[1] = 0.67

    optimizer = fitting.Adam(alpha=0.05)

    opt = optimizer.minimize(
        fitting.test_stress_parameters,
        curve,
        start_params,
        interior,
        constraints=constraints,
        max_iterations=iterations,
        verbose=verbose,
        batch_size=16
    )

    cols = ['loss', 'phase', 'obliquity']
    if paramCount == 3:
        cols.append('longitude')
    fitFrame = pd.DataFrame(opt[3],
                            columns=cols).sort_values('loss')[0:number_of_fits]
    fitFrame['FitNumber'] = range(1, number_of_fits + 1)

    fullCurve = analysisCurve if analysisCurve is not None else curve

    for fit in fitFrame.itertuples():
        params = [fit.phase, fit.obliquity]
        if paramCount == 3:
            params.append(fit.longitude)
        bestFit = analyze_params(np.array(params),
                                 fullCurve,
                                 0.25,
                                 name,
                                 verbose,
                                 folder,
                                 suffix=fit.FitNumber)
        bestFit.to_csv(f'{folder}{name}BestFit{fit.FitNumber}.csv.gz',
                       index=False, encoding='utf-8', compression='gzip')

    df = pd.DataFrame(opt[3], columns=cols).copy()

    df['phase'] = df['phase'] * (max_vals[0] - min_vals[0]) + min_vals[0]
    df['obliquity'] = df['obliquity'] * (max_vals[1] - min_vals[1]) + min_vals[1]
    if paramCount == 3:
        df['longitude'] = df['longitude'] * (max_vals[2] - min_vals[2]) + min_vals[2]

    df.to_csv(f'{folder}{name}Fits.csv.gz', index=False, encoding='utf-8', compression='gzip')

    
def process_cycloid_by_name(name, folder='./output/', iterations=3000, constraints=constraints, verbose=False):
    process_cycloid(cycloids[name].curve,
                    name,
                    analysisCurve=highResCycloids[name].curve,
                    folder=folder,
                    iterations=iterations,
                    constraints=constraints,
                    verbose=verbose)

## Generate Cyloid Data in Full Obliquity Range

In [5]:
for key in cycloids:
    process_cycloid_by_name(key, 
                            iterations=2000,
                            folder='./output/fullObliquityRange/',
                            verbose=False)

[08:49:14] Processing Cycloid: alex
Iteration 150/2000 -- Loss Output: 8.920044958005164 -- Moving Avg Loss: 10.432709334162006
	Parameters used: [0.10877435 0.30438117]
Iteration 300/2000 -- Loss Output: 1.1227306652741154 -- Moving Avg Loss: 1.282155248409742
	Parameters used: [0.53562021 0.87363327]
Iteration 450/2000 -- Loss Output: 0.7325965639744422 -- Moving Avg Loss: 2.415568040573023
	Parameters used: [0.62324668 0.14296286]
Iteration 600/2000 -- Loss Output: 1.1741233963325186 -- Moving Avg Loss: 1.4061807502154846
	Parameters used: [0.69809626 0.18553366]
Iteration 750/2000 -- Loss Output: 0.8342071991144359 -- Moving Avg Loss: 0.3722574268983112
	Parameters used: [0.84131224 0.55522016]
Iteration 900/2000 -- Loss Output: 0.0486009826545592 -- Moving Avg Loss: 0.09531378243667145
	Parameters used: [0.79650552 0.51109519]
Iteration 1050/2000 -- Loss Output: 0.2998940779298645 -- Moving Avg Loss: 0.4690945467702166
	Parameters used: [0.67010518 0.38668292]
Iteration 1200/2000 

## Non-Optimized Fits 

* 0.25 Deg Obliquity
* Phases 0, 60, 120, 180, 240 and 300 degrees

In [6]:
losses = []
folder = './output/lockedFits/'

OBLIQUITY = 0.25
phases = [0, 60, 120, 180, 240, 300]

for current in cycloids:
    for phase in phases:
        print(f'Processing phase {phase}')
        data, loss = direct_fit(cycloids[current].curve, 
                                0.25, 
                                show_plots=False, 
                                params=[phase, OBLIQUITY, 0])

        losses.append(dict(cycloid=current, loss=loss, phase=phase))
        print(current,'\t-', loss)

        filename = f'{folder}{current}-phase{phase}.csv.gz'
        data.to_csv(filename, index=False, compression='gzip')

lossFrame = pd.DataFrame(losses)
filename = f'{folder}Losses.csv'
lossFrame.to_csv(filename, index=False)

Processing phase 0
alex 	- 1.101083245680132
Processing phase 60
alex 	- 1.4971956846431838
Processing phase 120
alex 	- 0.6598343140300722
Processing phase 180
alex 	- 0.7702316207406991
Processing phase 240
alex 	- 0.43309652602466875
Processing phase 300
alex 	- 0.5071788920359364
Processing phase 0
carly 	- 0.35328613497182065
Processing phase 60
carly 	- 0.21267623114570441
Processing phase 120
carly 	- 0.24922552695109376
Processing phase 180
carly 	- 0.538335195440918
Processing phase 240
carly 	- 0.8777948253803868
Processing phase 300
carly 	- 1.8209416009303054
Processing phase 0
cilicia 	- 0.012293092595647051
Processing phase 60
cilicia 	- 0.01239612808496264
Processing phase 120
cilicia 	- 0.012877744702335128
Processing phase 180
cilicia 	- 0.013864483392336313
Processing phase 240
cilicia 	- 0.011287991762470551
Processing phase 300
cilicia 	- 0.011243036088465809
Processing phase 0
delphi 	- 0.013205383710725268
Processing phase 60
delphi 	- 0.013160345707482517
Process

## Direct Fits (no phase or obliquity) for all cycloids

In [7]:
losses = []
folder = './output/directFits/'

for current in cycloids:
    data, loss = direct_fit(cycloids[current].curve, 0.25, show_plots=False)

    losses.append(dict(cycloid=current, loss=loss))
    print(current,'\t-', loss)

    filename = f'{folder}{current}.csv.gz'
    data.to_csv(filename, index=False, compression='gzip')

lossFrame = pd.DataFrame(losses)
filename = f'{folder}directLosses.csv'
lossFrame.to_csv(filename, index=False)

alex 	- 0.90675499538864
carly 	- 0.4679169956204177
cilicia 	- 0.011850136742198897
delphi 	- 0.01290892547457404
dirk 	- 2.1386070161654858
mira 	- 2.261342358536797
odessa 	- 0.6678619449895173
sidon 	- 0.010174794559418736
tyrrel 	- 0.8524575485454637
yaphet 	- 0.21000699087411415


## Fits with Tight Obliquity

In [8]:
for cycloid in cycloids:
    process_cycloid_top_fits(cycloids[cycloid].curve,
                             cycloid,
                             folder='./output/tightObliquity/',
                             iterations=2000,
                             constraints=tight_obliquity_contstraints,
                             number_of_fits=5,
                             paramCount=2,
                             verbose=False
                            )

Iteration 150/2000 -- Loss Output: 0.06210398648378454 -- Moving Avg Loss: 0.0563501429239001
	Parameters used: [0.74296968 0.74300612]
Iteration 300/2000 -- Loss Output: 0.055815900385971715 -- Moving Avg Loss: 0.07943345115011605
	Parameters used: [0.69584328 0.76574406]
Iteration 450/2000 -- Loss Output: 0.09070572030638192 -- Moving Avg Loss: 5.075391706113116
	Parameters used: [0.63988126 0.65579232]
Iteration 600/2000 -- Loss Output: 0.05964518576883631 -- Moving Avg Loss: 0.08081977253779796
	Parameters used: [0.69372664 0.75877465]
Iteration 750/2000 -- Loss Output: 0.06700438921192332 -- Moving Avg Loss: 0.11248966027489621
	Parameters used: [0.67797218 0.68870854]
Iteration 900/2000 -- Loss Output: 10.408914424168042 -- Moving Avg Loss: 3.853511851566286
	Parameters used: [0.25162284 0.66346267]
Iteration 1050/2000 -- Loss Output: 0.042283721541265105 -- Moving Avg Loss: 0.05482456510537623
	Parameters used: [0.72102838 0.81522621]
Iteration 1200/2000 -- Loss Output: 0.072036

## Fits with Specific Obliquity

In [10]:
OBLIQUITY = 0.25
constraints = getConstraints(OBLIQUITY, longitude_max=0)

for cycloid in cycloids:
    process_cycloid_top_fits(cycloids[cycloid].curve,
                             cycloid,
                             folder='./output/lockedObliquity025/',
                             iterations=2000,
                             constraints=constraints,
                             number_of_fits=5,
                             paramCount=2,
                             verbose=False
                            )
    
# process_cycloid_top_fits(cycloids['alex'].curve,
#                          cycloid,
#                          folder='./output/lockedObliquity025/',
#                          iterations=2000,
#                          constraints=constraints,
#                          number_of_fits=5,
#                          paramCount=2,
#                          verbose=False
#                         )

Iteration 150/2000 -- Loss Output: 2.16024819583002 -- Moving Avg Loss: 1.863116178481181
	Parameters used: [0.64146317 0.25      ]
Iteration 300/2000 -- Loss Output: 3.1661546059353585 -- Moving Avg Loss: 3.136314110969231
	Parameters used: [0.48571932 0.25      ]
Iteration 450/2000 -- Loss Output: 0.2787510925695823 -- Moving Avg Loss: 1.148028478119953
	Parameters used: [0.6975672 0.25     ]
Iteration 600/2000 -- Loss Output: 0.20185797208509196 -- Moving Avg Loss: 1.1353190246828213
	Parameters used: [0.74479543 0.25      ]
Iteration 750/2000 -- Loss Output: 2.793670412617595 -- Moving Avg Loss: 1.4562211783827896
	Parameters used: [0.5776587 0.25     ]
Iteration 900/2000 -- Loss Output: 3.597043075399308 -- Moving Avg Loss: 2.5299871560869676
	Parameters used: [0.4706108 0.25     ]
Iteration 1050/2000 -- Loss Output: 0.4135257418138971 -- Moving Avg Loss: 1.052964572909955
	Parameters used: [0.68564265 0.25      ]
Iteration 1200/2000 -- Loss Output: 11.612648662229123 -- Moving Av